# Debugging Jupyter Notebook for OCTolyzer's analysis of H-line/V-line single OCT B-scans

This notebook copies the step-by-step process of OCTolyzer's analysis pipeline for single H-line/V-line OCT B-scan analysis, and should provide the end-user with the means of debugging the pipeline by testing each step individually.

### Add OCTolyzer to system paths to permit access to analysis files

In [ ]:
import sys
from importlib import reload
sys.path.append(r'../')

### Import necessary packages and libraries

In [ ]:
import numpy as np
import os
import cv2
import pandas as pd
import scipy
import shutil
import matplotlib.pyplot as plt
from octolyzer import utils, analyse, analyse_slo, main
from octolyzer.segment.octseg import choroidalyzer_inference, deepgpet_inference
from octolyzer.segment.sloseg import slo_inference, avo_inference, fov_inference
from octolyzer.measure.bscan.thickness_maps import grid, map as map_module
from octolyzer.measure.bscan import bscan_measurements, utils as chor_utils

### Define the file you want to use to debug, the `path` is the only variable to change

In [ ]:
# path = r"../demo/input/Radial_1.vol"
path = r"../demo/input/Linescan_1.vol"

In [ ]:
fname = os.path.split(path)[1].split(".")[0]

save_path = f"check_linescan_radial/{fname}"
segmentation_directory = f"check_linescan_radial/oct_segmentations"

if not os.path.exists(os.path.split(save_path)[0]):
    os.mkdir(os.path.split(save_path)[0])

if not os.path.exists(save_path):
    os.mkdir(save_path)

if not os.path.exists(segmentation_directory):
    os.mkdir(segmentation_directory)

## Set the relevant configuration parameters in `param_dict` to whatever is relevant to your analysis

In [ ]:
param_dict = {
    "save_individual_segmentations": 1,
    "save_individual_images": 1,
    "preprocess_bscans": 1,
    "analyse_choroid": 1,
    "analyse_slo": 1,
    "custom_maps": [], # this cannot be "0" like it is in config.txt - it is an empty list
    "analyse_all_maps": 1,
    "analyse_square_grid": 0,
    "choroid_measure_type": "perpendicular",
    "linescan_roi_distance": 2000
}

# flags for choroid analysis, preprocessing bscans
preprocess_data = param_dict["preprocess_bscans"]

# For saving out representative Bscan/SLO/segmentation masks
save_ind_segmentations = param_dict["save_individual_segmentations"]
save_ind_images = param_dict["save_individual_images"]

# Custom retinal thickness maps
custom_maps = param_dict["custom_maps"]
all_maps = param_dict["analyse_all_maps"]

# analysing choroid?
analyse_choroid = param_dict['analyse_choroid']

# square grid for Ppole
sq_grid_flag = param_dict['analyse_square_grid']

# analysing SLO?
analyse_slo_flag = param_dict['analyse_slo']

# User-specified measure type for choroid
chor_measure_type = param_dict['choroid_measure_type']

# User-specified ROI distance either side of fovea
macula_rum = param_dict['linescan_roi_distance']

In [ ]:
# By default we save individual results and collate segmentations
collate_segmentations = 1

# Default bscan/slo measurement parameters
N_measures = "all" # Measuring all thicknesses across ROI to average over
N_avgs = 0 # Robust thickness estimation, only relevant when N_measures is an integer
chor_linescan_measure_type = chor_measure_type # Measuring type for choroidal metrics across OCT Linescans
chor_ppole_measure_type = chor_measure_type # Measuring type for choroidal metrics across OCT Volumes
ret_measure_type = 'vertical' # Measuring retina column-wise (via A-scans) according to most devices/literature


### Load in data from `.vol` file

In [ ]:
verbose=1
oct_output = []
logging_list=[]
have_slo = True

choroidalyzer = None
deepgpet = None

In [ ]:
output = utils.load_volfile(path, preprocess=preprocess_data*analyse_choroid, verbose=verbose,
                            custom_maps=custom_maps, logging=logging_list)
bscan_data, metadata, slo_output, layer_pairwise, logging_list = output
(slo, slo_acq_fixed, slo_acq, (slo_pad_x, slo_pad_y)) = slo_output
slo_pad_xy = np.array([slo_pad_x[0], slo_pad_y[0]])
N_scans, M, N = bscan_data.shape
slo_N = slo.shape[0]
oct_output.append(bscan_data)

# Pixel spacing, SLO pixel scaling is assumed as isotropic
scaleX, scaleY, scaleZ = metadata["bscan_scale_x"],metadata["bscan_scale_y"],metadata["bscan_scale_z"]
bscan_scale = (scaleX, scaleY)
bscan_ROI = metadata['bscan_ROI_mm']
slo_scale = metadata["slo_scale_xy"]

# Analyse the SLO image
scan_type = metadata["bscan_type"]
scan_location = metadata["location"]
if scan_location == "peripapillary":
    slo_location = "Optic disc"
else:
    slo_location = "Macula"
eye = metadata["eye"]

output[1]

In [ ]:
# Alter feature measurement distance for single/radial linescans based on bscan_ROI_mm
if scan_location == 'macular':
    roi_str = np.round(bscan_ROI, 3)

    # If line-scan oriented
    if scan_type in ['Radial', 'V-line', 'H-line']:

        # If the specified distance to measure is greater than 90% of the ROI captures on the B-scan
        # reduce to default 1500 microns either side of fovea.
        if 2*macula_rum > 1e3*bscan_ROI:
            mac_str = np.round(2*macula_rum/1e3, 3)
            msg = f"""\nB-scan ROI smaller than requested distance to measure ({mac_str}mm > {roi_str}mm). 
Reducing feature measurement distance to default value of 1500 microns either side of fovea."""
            logging_list.append(msg)
            if verbose:
                print(msg)
            macula_rum = 1500

### Load in SLO models and analyse SLO (assuming the SLO analysis is *not* where the error is coming from)

In [ ]:
slo_metrics = False
segmentation_dict = {}
slo_model = slo_inference.SLOSegmenter()
fov_model = fov_inference.FOVSegmenter()
avo_model = avo_inference.AVOSegmenter()
slo_analysis_output = analyse_slo.analyse(255*slo, save_path,
                                        slo_scale, slo_location, eye,
                                        slo_model, avo_model, fov_model,
                                        save_images=save_ind_segmentations, 
                                        compute_metrics=slo_metrics, verbose=verbose, 
                                        collate_segmentations=True, segmentation_dict=segmentation_dict)
slo_meta_df, slo_measure_dfs, _, slo_segmentations, slo_logging_list = slo_analysis_output
slo_missing_fovea = slo_meta_df.slo_missing_fovea.values[0].astype(bool)
logging_list.extend(slo_logging_list)
slo_avimout = slo_segmentations[-1]

### Load in choroid segmentation models and segment (if `analyse_choroid` is 1)

In [ ]:
# Alert to user we are analysing OCT from here on
msg = f"\n\nANALYSING OCT of {fname}.\n"
logging_list.append(msg)
if verbose:
    print(msg)

# Forcing model instantiation if unspecified
# Choroidalyzer for macular B-scans
if choroidalyzer is None or type(choroidalyzer) != choroidalyzer_inference.Choroidalyzer:
    msg = "Loading models..."
    logging_list.append(msg)
    if verbose:
        print(msg)
    choroidalyzer = choroidalyzer_inference.Choroidalyzer()
    
# DeepGPET for peripapillary B-scans
if deepgpet is None or type(deepgpet) != deepgpet_inference.DeepGPET:
    deepgpet = deepgpet_inference.DeepGPET()

# Segment choroid
# If macula-centred, use Choroidalyzer. If optic disc-centred, use deepGPET
scan_type = metadata["bscan_type"]
if analyse_choroid:
    msg = "Segmenting choroid and fovea..."
else:
    msg = "Detecting fovea for grid/ROI alignment (through use of Choroidalyzer)..."
logging_list.append(msg)
if verbose:
    print(msg)

if scan_location == "macular":
    if N_scans == 1 or choroidalyzer.device == 'cpu':
        rvfmasks, foveas, fov_scores = choroidalyzer.predict_list(bscan_data, soft_pred=True)
    else:
        rvfmasks, foveas, fov_scores = choroidalyzer.predict_batch(bscan_data, soft_pred=True)
elif scan_location == "peripapillary":
    rvfmasks = deepgpet.predict_list(bscan_data, soft_pred=True)

# Resolve fovea detection. If at origin then threshold too high, apply filter function and warn user.
if scan_location != "peripapillary":
    fovea_slice_num = N_scans//2 #fov_scores.mean(axis=1).argmax()
    fovea = foveas[fovea_slice_num]

In [ ]:
# Detect retinal layer keys
pairwise_keys = list(layer_pairwise.keys())
layer_keys = list(set(pd.DataFrame(pairwise_keys).reset_index(drop=True)[0].str.split("_", expand=True).values.flatten()))

### Post-process segmentation and add to retinal layer segmentations

In [ ]:
# Unpack segmentations if analysing choroid
if analyse_choroid:
    
    # Extract region mask and remove any vessel segmented pixels from outside segmented choroid
    traces = []
    rmasks = []
    vmasks = []
    vcmaps = []
    for rvf in rvfmasks:
        trace = utils.get_trace(rvf[0], 0.5, align=True)
        rmask = utils.rebuild_mask(trace, img_shape=(M, N))
        vmask = rmask.astype(float) * rvf[1].astype(float)
        vcmap = np.concatenate([vmask[...,np.newaxis]] 
                                + 2*[np.zeros_like(vmask)[...,np.newaxis]] 
                                + [vmask[...,np.newaxis] > 0.01], axis=-1)
        traces.append(np.array(trace))
        rmasks.append(rmask)
        vmasks.append(vmask)
        vcmaps.append(vcmap)

    # Add choroid layer segmentation key 
    layer_pairwise["CHORupper_CHORlower"] = traces
    layer_keys.append("CHORupper")
    layer_keys.append("CHORlower")

### Compute thickness, area, vascular index on the B-scan

In [ ]:
# Analysis isn't entirely supported yet for AV-line scans, as they are not fixed around the fovea,
# so just save out Bscan, SLO and the segmentations, do not measure.
if scan_type != "AV-line":

    # For a sequence of B-scans, measure thickness and area of all layers, and CVI for choroid
    msg = f"""Measuring average and subfoveal thickness, area, and vessel area/vascular index (for choroid only).
Region of interest is fovea-centred using a distance of {macula_rum}microns temporal/nasal to fovea.
All retinal measurements are made vertically, i.e. with respect to the image axis (vertical).
All choroidal measurements are made {chor_measure_type}."""
    logging_list.append(msg)
    if verbose:
        print(msg)

    # Collect measurements and ROI overlays per B-scan
    measure_dict = {}
    overlays = {'areas':[], 'thicks':[], 'macula_rum':macula_rum}
    for i in range(N_scans):
        msg = f"B-scan {i+1}:"
        if verbose:
            print(msg)

        # If we have identified the fovea, process measurements per layer
        measure_dict[i] = {}
        if foveas[i][0].sum() != 0:
            areas_to_overlay = ['ILM_BM']
            overlay_areas = []
            overlay_thicks = []

            # Loop over layers
            for key, tr in layer_pairwise.items():
                vess_mask = None
                meas_type = ret_measure_type
                if "CHOR" in key:
                    areas_to_overlay.append('CHORupper_CHORlower')
                    vess_mask = vmasks[i]
                    meas_type = chor_linescan_measure_type

                # Logging
                msg = f"    Measuring layer: {key}"
                logging_list.append(msg)
                if verbose:
                    print(msg)

                # Compute measurements 
                output, plotinfo, bscan_log = bscan_measurements.compute_measurement(tr[i], 
                                                                                     vess_mask=vess_mask, 
                                                                                     fovea=foveas[i], 
                                                                                     scale=bscan_scale, 
                                                                                     macula_rum=macula_rum, 
                                                                                     N_measures=N_measures, 
                                                                                     N_avgs=N_avgs,
                                                                                     measure_type=meas_type, 
                                                                                     img_shape=(M,N),
                                                                                     verbose=True, 
                                                                                     force_measurement=False, 
                                                                                     plottable=True, 
                                                                                     logging_list=[])
                logging_list.extend(bscan_log)

                # Append dictionary of measurements per layer per B-scan
                measure_dict[i][key] = {"subfoveal_thickness_[um]":output[0], "thickness_[um]":output[1], "area_[mm2]":output[2]}
                if "CHOR" in key:
                    measure_dict[i][key]["vascular_index"] = output[3]
                    measure_dict[i][key]["vessel_area_[mm2]"] = output[4]

                # Append ROI overlays per layer
                if key in areas_to_overlay:
                    if plotinfo is not None:
                        overlay_areas.append(plotinfo[1])
                        overlay_thicks.append(plotinfo[0][[0,-1]][:,0])
                    else:
                        overlay_areas.append(np.zeros_like(rmasks[0]))
                        overlay_thicks.append(None)

            # Append to outer list per B-scan
            overlays['areas'].append(overlay_areas)
            overlays['thicks'].append(overlay_thicks)
            
        else:

            # Warn user 
            msg = """Warning: The fovea has not been detected on the OCT B-scan.
This could be because the fovea is not present in the scan, or because of a segmentation error.
Skipping file and outputting -1s for measurements of each layer."""
            logging_list.append(msg)
            if verbose:
                print(msg)

            # Populate measurement dictionary with -1s
            for key, tr in layer_pairwise.items():
                measure_dict[i][key] = {"subfoveal_thickness_[um]":-1, "thickness_[um]":-1, "area_[mm2]":-1}
                
                # Explicitly add vessel area and CVI to measure_dict for choroid
                if "CHOR" in key:
                    measure_dict[i][key]["vascular_index"] = -1
                    measure_dict[i][key]["vessel_area_[mm2]"] = -1

                # Add in dummy ROI maps to ensure plot_composite_bscans(...) still runs
                if key in areas_to_overlay:
                    overlay_areas.append(np.zeros_like(rmasks[0]))
                    overlay_thicks.append(None)

            # Append to outer list per B-scan
            overlays['areas'].append(overlay_areas)
            overlays['thicks'].append(overlay_thicks)


    # Stitch all B-scans to create "contact sheet" for checking
    # this is compatible for single linescans, radial scans and volume scans.
    if N_scans in [1,6,8,10,12]:
        if N_scans == 1:
            reshape_idx = (1,1)
        elif N_scans == 6:
            reshape_idx = (2,3) 
        elif N_scans == 8:
            reshape_idx = (2,4)
        elif N_scans == 10:
            reshape_idx = (2,5)
        elif N_scans == 12:
            reshape_idx = (3,4)
        utils.plot_composite_bscans(bscan_data, 
                                    vmasks, 
                                    foveas, 
                                    layer_pairwise, 
                                    reshape_idx, 
                                    analyse_choroid, 
                                    fname, 
                                    save_path, 
                                    overlays)
        
        # Copy composite into oct_segmentations directory
        if collate_segmentations:
            shutil.copy(os.path.join(save_path, f"{fname}_linescan_octseg.png"),
                        os.path.join(segmentation_directory, f"{fname}.png"))
    
    else:
        msg = f'Radial scan pattern with {N_scans} B-scans cannot currently be reshaped into single, composite image.\nThis is likely because the development team has not had access to this kind of radial scan before. Please raise an issue on the GitHub repository.'
        logging_list.append(msg)
        if verbose:
            print(msg)
            
# If AV-line scan no feature measurement or saving out of segmentations
elif scan_type == "AV-line":
    msg = f"""Scan location intersects arteries/veins and is not fovea-centred OR acquisition line is not horizontal/vertical.
Measurements of thickness, area, etc. are not supported (yet).
Instead, B-scan and SLO images are automatically saved out."""
    logging_list.append(msg)
    if verbose:
        print(msg)
    measure_dict = {}
    save_ind_images = 1

### Organise metadata

In [ ]:
# H-line/V-line/Radial Measurement metadata
horizontal = [False, True][scan_type in ["H-line", "Radial"]]
if scan_type in ["H-line", "V-line", "Radial"]:
    metadata["bscan_missing_fovea"] = False
    metadata["slo_missing_fovea"] = False

    # Save out fovea xy-coordinates, comma-separated for when N_scans > 1
    metadata["bscan_fovea_x"] = ','.join([f'{fov[0]}' for fov in foveas])
    metadata["bscan_fovea_y"] = ','.join([f'{fov[1]}' for fov in foveas])

    # Flag any missing fovea xy-coordinates
    if np.any(np.sum(np.array(foveas), axis=1) == 0):
        metadata["bscan_missing_fovea"] = True

    # SLO metadata on fovea
    if have_slo:

        # If we can identify fovea on B-scan, use this to cross-reference fovea on SLO
        output = map_module.detect_angle(slo_acq_fixed, 
                                         slo_pad_xy,
                                         fovea_slice_num,
                                         fovea=foveas[fovea_slice_num], 
                                         oct_N=N,
                                         horizontal=horizontal,
                                         N_scans=N_scans)
        acq_angle, fovea_at_slo_from_bscan, _, _ = output

        # Overwrite SLOctolyzer's fovea_at_slo with cross-referenced fovea_at_slo_from_bscan
        # if could not identify fovea on B-scan
        if fovea_at_slo_from_bscan.sum() != 0:
            fovea_at_slo = fovea_at_slo_from_bscan

        # If fovea detection on B-scan failed and if we don't have fovea on SLO then 
        # resort to np.array([0,0])
        else:
            if not analyse_slo_flag:
                fovea_at_slo = fovea_at_slo_from_bscan
                metadata["slo_missing_fovea"] = True 

        # Append fovea on SLO to metadata, updating angle
        metadata["slo_fovea_x"] = fovea_at_slo[0]
        metadata["slo_fovea_y"] = fovea_at_slo[1]

        # Acquisition angle for single linescan (H-line: 0 degrees, V-line: 90 degrees)
        if scan_type != "Radial":
            metadata["acquisition_angle_degrees"] = str(acq_angle)

        # For radial scan, create list of angles from H-line. Scan 0 is always V-line (90-degrees 
        # and rotates at even intervals)
        else:
            metadata["acquisition_angle_degrees"] = ','.join([str(int(90-i*(360/(2*N_scans)))) for i in range(N_scans)])

    # ROI metadata
    metadata["linescan_area_ROI_microns"] = macula_rum
    metadata["choroid_measure_type"] = chor_measure_type

    # Missing measurements flagging
    metadata["missing_retinal_oct_measurements"] = False
    metadata["missing_choroid_oct_measurements"] = False
    for i in range(N_scans):
        img_measures = measure_dict[i]
        for key in pairwise_keys:
            if img_measures[key]["subfoveal_thickness_[um]"] == -1:
                metadata["missing_retinal_oct_measurements"] = True
                break 
            if "CHORupper_CHORlower" in list(img_measures.keys()):
                if img_measures["CHORupper_CHORlower"]["subfoveal_thickness_[um]"] == -1:
                    metadata["missing_choroid_oct_measurements"] = True

# If AV-line scan, assume fovea information unknown
else:
    metadata["bscan_fovea_x"] = None
    metadata["bscan_fovea_y"] = None
    metadata["slo_fovea_x"] = None
    metadata["slo_fovea_y"] = None
    metadata["acquisition_angle_degrees"] = None

In [ ]:
metadata

### Save out SLO and B-scan images

In [ ]:
# If saving out bscan and slo image. If ppole, only saying out bscan at fovea
# This is automatically done for AV-line scans.
if save_ind_images:
    if have_slo:
        cv2.imwrite(os.path.join(save_path,f"{fname}_slo.png"), 
                    (255*slo).astype(np.uint8))
    if scan_location != 'peripapillary':
        cv2.imwrite(os.path.join(save_path,f"{fname}_slo_acquisition_lines.png"), 
                    (255*slo_acq).astype(np.uint8))
        cv2.imwrite(os.path.join(save_path,f"{fname}_bscan_fovea.png"), 
                (255*bscan_data[fovea_slice_num]).astype(np.uint8))
    else:
        cv2.imwrite(os.path.join(save_path,f"{fname}_bscan.png"), 
                (255*bscan_data[0]).astype(np.uint8))
        

# Save out raw probability vessel segmentation maps if analysing choroid and analysing peripapillary scan
if scan_location != 'peripapillary':
    if save_ind_segmentations and analyse_choroid:
        if N_scans == 1:
            cv2.imwrite(os.path.join(save_path, f"{fname}_chorvessel_mask.png"), (255*vmasks[fovea_slice_num]).astype(int))
        else:
            np.save(os.path.join(save_path, f"{fname}_chorvessel_maps.npy"), vmasks)

### Organise measurements into DataFrame

In [ ]:
KEY_LAYER_DICT = {"ILM": "Inner Limiting Membrane",
                  "RNFL": "Retinal Nerve Fiber Layer",
                  "GCL": "Ganglion Cell Layer",
                  "IPL": "Inner Plexiform Layer",
                  "INL": "Inner Nuclear Layer",
                  "OPL": "Outer Plexiform Layer",
                  "ELM": "External Limiting Membrane", # Outer nuclear layer
                  "PR1": "Photoreceptor Layer 1",
                  "PR2": "Photoreceptor Layer 2",
                  "RPE": "Retinal Pigment Epithelium",
                  "BM": "Bruch's Membrane Complex", 
                  "CHORupper": "Bruch's Membrane - Choroid boundary",
                  "CHORlower": "Choroid - Sclera boundary"}

In [ ]:
# For H-line/V-line/Radial organise feature measurements into a DataFrame
if scan_type != "AV-line":
    measure_df = utils.nested_dict_to_df(measure_dict).reset_index()
    if scan_type != 'Peripapillary':
        measure_df = measure_df.rename({"level_0":"scan_number", "level_1":"layer"}, inplace=False, axis=1)
    else:
        measure_df = measure_df.rename({"index":"layer"}, inplace=False, axis=1)
    
    # rename whole/inner/outer retinal layers
    keys_to_names = ['ILM_BM', 'ILM_ELM', 'ELM_BM']
    names_to_keys = ['retina', 'inner_retina', 'outer_retina']
    for k2n, n2k in zip(keys_to_names, names_to_keys):
        measure_df.replace(k2n, n2k, inplace=True)
    
    # order map layer names anatomically
    all_pairwise_layers = list(layer_pairwise.keys())
    all_pairwise_layers = names_to_keys + all_pairwise_layers
    ordered_maps = []
    for map_name in all_pairwise_layers:
        if map_name in list(measure_df.layer.values):
            ordered_maps.append(map_name)
    measure_df['layer'] = pd.CategoricalIndex(measure_df['layer'], ordered=True, categories=ordered_maps)
    measure_df = measure_df.sort_values('layer').reset_index(drop=True)
    measure_dfs = [measure_df]

# Layer keys, ordered anaomtically
ordered_keys = np.array(list(KEY_LAYER_DICT))
key_df = pd.DataFrame({"key":layer_keys,"layer":[KEY_LAYER_DICT[key] for key in layer_keys],
                    "layer_number":[np.where(key == ordered_keys)[0][0] for key in layer_keys]})
key_df = key_df.sort_values("layer_number")
del key_df["layer_number"]

### Sort segmentations to be saved out

In [ ]:
# Organise layer segmentations to be saved out - overcomplicated as I am working
# with pairwise segmentation traces, not individual ones. 
seg_df = {}
layer_keys_copied = layer_keys.copy()
for key, trace_xy_all in layer_pairwise.items():
    for k_idx, k in enumerate(key.split("_")):
        if k in layer_keys_copied:
            all_ytr = {}
            for s_idx, trace in enumerate(trace_xy_all):
                t = trace[k_idx]
                (xtr, ytr) = t[:,0], t[:,1]
                try:
                    xst, xen = xtr[[0,-1]]
                    ytr_pad = np.pad(ytr, ((max(xst-1,0), N-xen)), mode="constant")
                    all_ytr[s_idx] = {i:ytr_pad[i] for i in range(N)}
                except Exception as e:
                    message = f"\nAn exception of type {type(e).__name__} occurred. Error description:\n{e.args[0]}"
                    user_fail = f"Failed to store segmentations for B-scan {s_idx+1}/{N_scans} for layer {k}. Saving as NAs"
                    log_save = [message, user_fail]
                    logging_list.extend(log_save)
                    if verbose:
                        print(message)
                        print(user_fail)
                    all_ytr[s_idx] = {i:np.nan for i in range(N)}

            layer_keys_copied.remove(k)
            df = utils.nested_dict_to_df(all_ytr).reset_index()
            df = df.rename({"index":"scan_number"}, inplace=False, axis=1)
            seg_df[k] = df

### Save out results

In [ ]:
# Save out core results in an .xlsx file
meta_df = pd.DataFrame(metadata, index=[0])
with pd.ExcelWriter(os.path.join(save_path, f'{fname}_output.xlsx')) as writer:
    
    # Write metadata
    meta_df.to_excel(writer, sheet_name='metadata', index=False)

    # Save out metadata key and descriptions
    metakeydf = utils.metakey_df
    metakeydf = metakeydf[metakeydf.column.isin(list(meta_df.columns))]
    metakeydf.to_excel(writer, sheet_name='metadata_keys', index=False)

    # Write OCT results, either map measurements (for PPole only) or H-line/V-line/Radial measurements
    if scan_type == "Ppole":
        for measure_df, volmeasure_df, grid_type in zip(measure_dfs, volmeasure_dfs, measure_grids):
            measure_df.to_excel(writer, sheet_name=f'{grid_type}_measurements', index=False)
            volmeasure_df.to_excel(writer, sheet_name=f'{grid_type}_volume_measurements', index=False)    
    elif scan_type != "AV-line":
        for measure_df in measure_dfs:
            measure_df.to_excel(writer, sheet_name="oct_measurements", index=False)

    # Write SLO measurements
    if slo_analysis_output is not None and analyse_slo:
        for df in slo_measure_dfs:
            if len(df) > 0:
                z = df.zone.iloc[0]
                df.to_excel(writer, sheet_name=f'slo_measurements_{z}', index=False)

    # Write out segmentations
    for key,df in seg_df.items():
        if key != key.lower():
            name = f"segmentations_{key}"
        else:
            name = f"maps_{key}"
        df.to_excel(writer, sheet_name=name, index=False)

    # write out layer keys
    key_df.to_excel(writer, sheet_name="layer_keys", index=False)

msg = f"\nSaved out metadata, measurements and segmentations."
logging_list.append(msg)
if verbose:
    print(msg)

# Organise outputs from analysis script
oct_measures = measure_dfs.copy()
oct_segmentations_maps = [seg_df]
if scan_location != 'peripapillary':
    oct_segmentations_maps.append(vmasks)
if scan_type == "Ppole":
    oct_segmentations_maps.append(map_dict)
    for df in volmeasure_dfs:
        oct_measures.append(df)
    if sq_grid_flag:
        oct_measures = [oct_measures[0],oct_measures[2],oct_measures[1],oct_measures[3]]
oct_analysis_output = [meta_df, slo, bscan_data] + [oct_measures] + oct_segmentations_maps + [logging_list]

# final log
msg = f"\nCompleted analysis of {fname}.\n"
logging_list.append(msg)
if verbose:
    print(msg)

# Save out log
with open(os.path.join(save_path, f"{fname}_log.txt"), "w") as f:
    for line in logging_list:
        f.write(line+"\n")

# Run OCTolyzer's pipeline from scratch on this file to check whether it works properly before/after debugging. 

In [ ]:
analyse.analyse(path, save_path="check_linescan_radial", param_dict=param_dict)